In [2]:
import boto3
import pandas as pd
from io import StringIO

In [6]:
s3=boto3.resource('s3')
bucket=s3.Bucket('xetra-1234')

In [8]:
bucket_obj1 = bucket.objects.filter(Prefix='2022-01-27/')
bucket_obj2 = bucket.objects.filter(Prefix='2022-01-28/')

objects=[obj for obj in bucket_obj1]+[obj for obj in bucket_obj2]

In [9]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-01-27/2022-01-27_BINS_XETR10.csv'),
 s3.Object

In [10]:
csv_object_init=bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data=StringIO(csv_object_init)
df_init=pd.read_csv(data, delimiter=',')

In [11]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [12]:
df_all=pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data=StringIO(csv_obj)
    df=pd.read_csv(data, delimiter=',')
    df_all = pd.concat([df_all, df], ignore_index=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11744\836491098.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df], ignore_index=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11744\836491098.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df], ignore_index=True)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11744\836491098.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a fu

In [13]:
columns=['ISIN','Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']
df_all=df_all.loc[:,columns]

In [14]:
df_all.dropna(inplace=True)

In [15]:
df_all.shape

(250384, 8)

# Get opening price per ISIN and day

In [21]:
df_all['opening_price']=df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')

In [17]:
'''df_all.sort_values(by=['Time']): Sorts the DataFrame by Time.
groupby(['ISIN', 'Date']): Groups the DataFrame by ISIN and Date.
transform('first'): Applies the first() function to each group, 
returning the first value in the StartPrice column for each group.'''

"df_all.sort_values(by=['Time']): Sorts the DataFrame by Time.\ngroupby(['ISIN', 'Date']): Groups the DataFrame by ISIN and Date.\ntransform('first'): Applies the first() function to each group, \nreturning the first value in the StartPrice column for each group."

# Get closing price per ISIN and day¶

In [22]:
df_all['closing_price']=df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')

In [19]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2022-01-27,08:00,16.35,16.40,16.35,16.38,7258,16.54
1,DE000A0D6554,2022-01-27,08:00,13.11,13.20,13.09,13.13,28264,13.48
2,DE000A0HN5C6,2022-01-27,08:00,35.39,35.39,35.39,35.39,98,35.74
3,DE000A0JL9W6,2022-01-27,08:00,50.20,50.25,49.26,49.34,19183,51.95
4,DE000A0LD2U1,2022-01-27,08:00,19.50,19.50,19.49,19.49,2537,19.50
...,...,...,...,...,...,...,...,...,...
250379,DE0007165631,2022-01-28,16:44,465.50,465.50,465.50,465.50,10,465.50
250380,DE000A2N4H07,2022-01-28,16:44,20.62,20.62,20.62,20.62,1,20.62
250381,LU1899270539,2022-01-28,16:52,107.50,107.50,107.50,107.50,48,107.50
250382,LU2249056297,2022-01-28,16:52,57.75,57.75,57.75,57.75,56,57.75


# Aggregations

In [23]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [25]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT000000STR1,2022-01-27,37.600,37.000,37.000,37.900,485
1,AT000000STR1,2022-01-28,38.050,37.000,37.000,38.050,456
2,AT00000FACC2,2022-01-27,7.640,7.620,7.620,7.640,60
3,AT00000FACC2,2022-01-28,7.660,7.520,7.520,7.660,610
4,AT0000606306,2022-01-27,24.160,25.080,24.160,25.260,1363
...,...,...,...,...,...,...,...
6360,XS2314659447,2022-01-28,8.774,8.576,8.570,8.774,1037
6361,XS2314660700,2022-01-27,20.206,20.316,20.206,20.472,148
6362,XS2314660700,2022-01-28,20.328,20.246,20.054,20.328,58
6363,XS2376095068,2022-01-27,32.216,32.884,32.216,33.326,11504


# percent change prev closing

In [26]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [27]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
df_all.drop(columns=['prev_closing_price'], inplace=True)